### ACO (Ant Colony Optimization) algorithm for Smart Grids

Intro

In [33]:
interval_num = 24
modes_num = 3
num_nodes = interval_num*modes_num

In [34]:
import networkx as nx
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
import random
from statistics import mean

In [35]:
np.seterr(divide='ignore', invalid='ignore')
None

### Generate random X and Y coordinates

In [36]:
xs = []
ys = []
pheromone = []
start_interval = -1
for _ in range(interval_num):
    start_interval += 1
    start_mode = 0
    for _ in range(modes_num):
        xs.append(start_interval)
        ys.append(start_mode)
        pheromone.append(1)
        start_mode += 1
print(xs)
print(ys)
schedule_map = list(zip(xs, ys, pheromone))

[0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 7, 8, 8, 8, 9, 9, 9, 10, 10, 10, 11, 11, 11, 12, 12, 12, 13, 13, 13, 14, 14, 14, 15, 15, 15, 16, 16, 16, 17, 17, 17, 18, 18, 18, 19, 19, 19, 20, 20, 20, 21, 21, 21, 22, 22, 22, 23, 23, 23]
[0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2]


### Generate the distances between all nodes

### ACO parameters

In [37]:
max_iterations = 100
num_ants = 20

rho = 0.05 # evaporation rate .. this determines how much pheromones are lost at each iteration
alpha = 1 # pheromone exponential parameter
beta = 1 # desirability exponential parameter

### Feasibility Function

In [38]:
cost_table = [300, 0, 0] # Mode 0 is fossil fuel, Mode 1 is battery, Mode 2 is Solar
generation_table = [15, 10, 5]
battery_capacity = 100 # kw/interval
battery_start = 100 # Is battery charged on schedule start?
unit_comsumption = 2
when_is_solar_available = [5-17]

def is_schedule_viable (actual_schedule):
    battery = battery_capacity
    actual_interval = 0
    for intervals in actual_schedule:
        if generation_table[intervals] < unit_comsumption:
            return False
        if intervals == 2 and intervals not in when_is_solar_available:
            return False
        else:
            return True
    actual_interval += 1


### ACO functions

In [39]:
def create_colony(num_ants):
    current_schedule_all = []
    for i in range(num_ants):
        current_schedule = []
        
        actual_interval = 0
        while actual_interval < interval_num:
            print(f'{actual_interval} intervals')
            possible_schedules = []
            possible_schedules_weight = []
            #print(schedule_map)
            for modes in schedule_map:
                #print(modes)
                if modes[0] == actual_interval:
                    #print(modes[1])
                    possible_schedules.append(modes[1])
                    #print(possible_schedules)
                    possible_schedules_weight.append(modes[2])
            #print(possible_schedules)
            #print(possible_schedules_weight)
            chosen_schedule = random.choices(possible_schedules, possible_schedules_weight)
            #print(chosen_schedule)
            current_schedule.append(chosen_schedule[0])
            print(is_schedule_viable(chosen_schedule))
            #print(possible_schedules)
            #print(current_schedule)
            actual_interval += 1
        
        current_schedule_all.append(current_schedule)
    
    return current_schedule_all
 

### Main loop of ACO

In [40]:
shortest_tour_length = float('inf')
all_solutions = np.empty((0, 2))

for i in range(0, max_iterations):
    # create the tours for all ants
    tours = create_colony(num_ants)

    # calculate the tour length for each tour
    tour_lengths = []
    for schedule in tours:
        total_cost = 0
        for interval_mode in schedule:
            total_cost += cost_table[interval_mode]
        print(total_cost)
        if total_cost <= shortest_tour_length:
            shortest_tour_length = total_cost
            best_tour = schedule
        tour_lengths.append(total_cost)
        tour_and_distance = list(zip((tours, tour_lengths)))
    #print(tour_and_distance)
    print(f'shortest tour {best_tour}')
    print(f'length {shortest_tour_length}')
        
    # update the pheromone matrix and evaporate
    interval_start = 0
    for interval in best_tour:
        for intervals in schedule_map:
            if intervals[0] == interval_start and intervals[1] == interval:
                intervals = (intervals[0], intervals[1], intervals[2]+alpha) 
            intervals = (intervals[0], intervals[1], intervals[2]*(1-rho))
        interval_start += 1


0 intervals
False
1 intervals
True
2 intervals
False
3 intervals
True
4 intervals
False
5 intervals
True
6 intervals
True
7 intervals
True
8 intervals
False
9 intervals
False
10 intervals
False
11 intervals
True
12 intervals
True
13 intervals
True
14 intervals
True
15 intervals
True
16 intervals
True
17 intervals
True
18 intervals
True
19 intervals
True
20 intervals
True
21 intervals
True
22 intervals
False
23 intervals
False
0 intervals
True
1 intervals
False
2 intervals
False
3 intervals
False
4 intervals
False
5 intervals
True
6 intervals
True
7 intervals
True
8 intervals
False
9 intervals
False
10 intervals
True
11 intervals
True
12 intervals
True
13 intervals
True
14 intervals
False
15 intervals
True
16 intervals
False
17 intervals
True
18 intervals
True
19 intervals
True
20 intervals
True
21 intervals
False
22 intervals
True
23 intervals
False
0 intervals
False
1 intervals
True
2 intervals
False
3 intervals
False
4 intervals
True
5 intervals
False
6 intervals
True
7 intervals
Tru

### Shortest Path Length